In [3]:
from torchaudio.datasets import LJSPEECH
from IPython.display import Audio

dataset = LJSPEECH(root="data", download=True)
waveform, sample_rate, transcript, normalized = dataset[76]

print(transcript)
wav_ref = waveform.squeeze().numpy()
Audio(wav_ref, rate=sample_rate)

In England about this time, an attempt was made (notably by Caslon, who started business in London as a type-founder in 1720)


In [1]:
import torch
from transformers import CsmForConditionalGeneration, AutoProcessor

model_id = "sesame/csm-1b"
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = AutoProcessor.from_pretrained(model_id)
model = CsmForConditionalGeneration.from_pretrained(model_id, device_map=device)

/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.41it/s]


In [4]:
import os, random

save_dir = "tts_out"
os.makedirs(save_dir, exist_ok=True)

random.seed(42)

N = len(dataset)
pick = random.sample(range(N), k=min(20, N))

In [5]:
for i in pick:
    trnascript = dataset[i]
    text = transcript
    
    conversation = [
        {
            "role": "0", 
            "content": [{"type": "text", "text": text}],
        },
    ]

    inputs = processor.apply_chat_template(
        conversation,
        tokenize=True,
        return_dict=True,
    ).to(device)
    
    audio = model.generate(**inputs, output_audio=True)
    out_path = os.path.join(save_dir, f"sample_{i:05d}.wav")
    processor.save_audio(audio, out_path)
    print(f"success to make sample_{i:05d}.wav")

success to make sample_10476.wav
success to make sample_01824.wav
success to make sample_00409.wav
success to make sample_12149.wav
success to make sample_04506.wav
success to make sample_04012.wav
success to make sample_03657.wav
success to make sample_02286.wav
success to make sample_12066.wav
success to make sample_01679.wav
success to make sample_11087.wav
success to make sample_12135.wav
success to make sample_08935.wav
success to make sample_01424.wav
success to make sample_09674.wav
success to make sample_06912.wav
success to make sample_00520.wav
success to make sample_00488.wav
success to make sample_01535.wav
success to make sample_03582.wav


In [6]:
sample_rate = 22050
#wav_np = (audio[0] if isinstance(audio, (list, tuple)) else audio).detach().cpu().numpy()
files = [f for f in os.listdir(save_dir) if f.endswith(".wav")]
wav_path = os.path.join(save_dir, files[6])
Audio(wav_path)

In [12]:
import numpy as np, librosa, pandas as pd

def lf0_rmse_refarray(ref_np, ref_sr, gen_wav_path, fmin=50.0, fmax=600.0, hop_length=256):
    try:
        y_gen, sr_gen = librosa.load(gen_wav_path, sr=ref_sr, mono=True)
    except Exception as e:
        print(f"Error loading {gen_wav_path}: {e}")
        return float("nan")

    f0_ref, _, _ = librosa.pyin(ref_np, fmin=fmin, fmax=fmax, sr=ref_sr, hop_length=hop_length)
    f0_gen, _, _ = librosa.pyin(y_gen,  fmin=fmin, fmax=fmax, sr=ref_sr, hop_length=hop_length)

    L = min(len(f0_ref), len(f0_gen))
    f0_ref_trunc = f0_ref[:L]
    f0_gen_trunc = f0_gen[:L]

    # 두 F0 시퀀스 모두 유성음인 프레임만 필터링
    voiced_mask = ~np.isnan(f0_ref_trunc) & ~np.isnan(f0_gen_trunc)
    
    # 0 Hz 이하의 값(F0 추출 오류)을 방지 (log 함수는 양수만 받음)
    valid_f0_mask = (f0_ref_trunc[voiced_mask] > 0) & (f0_gen_trunc[voiced_mask] > 0)
    
    f0_ref_voiced = f0_ref_trunc[voiced_mask][valid_f0_mask]
    f0_gen_voiced = f0_gen_trunc[voiced_mask][valid_f0_mask]

    if f0_ref_voiced.size == 0:
        return float("nan")

    log_f0_ref = np.log(f0_ref_voiced)
    log_f0_gen = np.log(f0_gen_voiced)
    
    # LF0 RMSE 계산 (단위는 log-Hz)
    # (선택) Cents 단위로 변환하려면: 1200 * np.log2(f_gen / f_ref)
    # 여기서는 SOTA 논문들처럼 log-Hz 단위를 사용합니다.
    lf0_rmse = np.sqrt(np.mean((log_f0_ref - log_f0_gen)**2))
    
    return float(lf0_rmse)


In [ ]:
for i in range(len(pick)):
    j = pick[i]          
    gen_path = os.path.join(save_dir, f"sample_{j:05d}.wav")

    waveform, sr_ref, _, _ = dataset[j]
    ref_np = waveform.squeeze().numpy()

    rmse = lf0_rmse_refarray(ref_np, sr_ref, gen_path)
    print(f"[{j}] F0 RMSE: {rmse:.2f} Hz")

[10476] F0 RMSE: 0.78 Hz
[1824] F0 RMSE: 0.20 Hz
[409] F0 RMSE: 0.32 Hz
[12149] F0 RMSE: 0.50 Hz
[4506] F0 RMSE: 0.52 Hz
[4012] F0 RMSE: 0.50 Hz
[3657] F0 RMSE: 0.26 Hz
[2286] F0 RMSE: 0.75 Hz
[12066] F0 RMSE: 0.51 Hz
[1679] F0 RMSE: 0.22 Hz
[11087] F0 RMSE: 0.66 Hz
[12135] F0 RMSE: 0.85 Hz
[8935] F0 RMSE: 0.24 Hz
[1424] F0 RMSE: 0.69 Hz
[9674] F0 RMSE: 0.41 Hz
[6912] F0 RMSE: 0.38 Hz
[520] F0 RMSE: 0.30 Hz
[488] F0 RMSE: 0.74 Hz
[1535] F0 RMSE: 0.75 Hz
